# Adding Axial Realism
* Up to this point all models were 1.26 cm tall with reflective boundaries
* Great for rough $k_\infty$, but not a useful $k_\infty$
* Lacks axial buckling, which may be of interest



## Goals
* Make assembly realistic
    * Expand to full assembly height
    * Add water reflectors at top and bottom
    * Remove axial reflective boundaries
    * Add Vacuum boundary cells
* Discretize axially
    * shrink lattice unit cell axially
    * repeat lattice axially
* Update `ksrc` to be physically possible

# Starting

* Load solution model `models/w17_17_kinf_ans.imcnp`: or, use your own from the previous exercise.

In [ ]:
import montepy
import numpy as np
from IPython.display import IFrame
import warnings

problem = montepy.read_input("models/w17_17_k_inf_ans.imcnp")
warnings.simplefilter("ignore", montepy.errors.LineExpansionWarning)

# Step 1: Update Axial Limits
## Step 1.1 Grab PZ planes
* Need to Grab the only two `PZ` surfaces in the model for update
* Fuel rod height: 384.70 cm<sup>1</sup>
* Can sort the surfaces by `surf.location` using [`sorted`](https://docs.python.org/3/library/functions.html#sorted) with `key`

<sup>1</sup> N. E. Horelik et al., "Benchmark for Evaluation and Validation of Reactor Simulations (BEAVRS)," presented at the Int. Conf. Mathematics and Computational Methods Applied to Nuc. Sci. & Eng., Sun Valley, Idaho, 2013.

In [ ]:
FUEL_ROD_HEIGHT = 384.70  # [cm]
WATER_DENSITY = 0.74  # [g/cm3]
sort_key = lambda s: s.location

# Task: Sort the Z-Planes by "location".

In [ ]:
# Task: Find 'pzs', the list of planes sorted from low to high.
pzs = None
#
bot = pzs[0]
top = pzs[-1]
print(repr(bot))
print(repr(top))

# Step 1.2 Update height.

1. Move datum to be at bottom of fuel pins
    * Set bottom plane to location of `0.0`
2. Move top to be the proper height
3. disable `is_reflecting`

In [ ]:
for surf in [bot, top]:
    surf.is_reflecting = False
bot.location = 0.0
top.location = FUEL_ROD_HEIGHT

# Step 2: Add water reflectors
## Step 2.1 Get all necessary objects
* Get water material
* Get bounding `px`, `py`
    * would be the only reflecting surfaces
 * Sort surfaces to get top/bottom, left/right

In [ ]:
water = list(problem.materials.get_containing_all("H", "O"))[0]
pxs = []
pys = []
print(water)
for surf in problem.surfaces.px:
    if surf.is_reflecting:
        pxs.append(surf)
for surf in problem.surfaces.py:
    if surf.is_reflecting:
        pys.append(surf)
# sort the surfaces
pxs = sorted(pxs, key=sort_key)
pys = sorted(pys, key=sort_key)
# Get x limits
left = pxs[0]
right = pxs[-1]
# get y limits
y_bot = pys[0]
y_top = pys[-1]
print(pxs)
print(pys)

# Step 2.2 Make bounding Top and Bottom surfaces
1. clone `pz` surfaces (`top`, `bot`)
2. Move to the appropriate location
3. Make reflector 20 cm thick
    * Can use `+=` and `-=` again.

In [ ]:
outer_top = top.clone()
outer_top.location += 20
outer_bot = bot.clone()
outer_bot.location -= 20
print(repr(outer_top))
print(repr(outer_bot))

# Step 2.3 Actually Make the Reflector cell

* Use union `|` to make only one cell

## Steps
1. Make the cell, number it, append it
2. Set geometry
   * Inside of the `x`, `y` boxes
   * either between the top reflect z-planes, or the bottom reflector z-planes
3. Assign `water` and `WATER_DENSITY`

# Task: Build the reflector cell

In [ ]:
# Task: Create the cell
cell = montepy.Cell(number=problem.cells.request_number())
# Task: Specify the region using the Union operator
cell.geometry = None

# Finish up
cell.material = water
cell.mass_density = WATER_DENSITY
print(cell.mcnp_str())

In [ ]:
problem.cells.append(cell)

# Step 3: Add vacuum boundary region.

* Reminder: in MCNP vacuum boundaries are defined by cells with 0 importance
* Will define a cell with 0 importance
* Can be an infinite cell
    * define as the union below the bottom, and above the top

## Steps
1. Make a cell
2. Define it's geometry

In [ ]:
# Make the "graveyard" cell
vacuum_cell = montepy.Cell(number=problem.cells.request_number())
vacuum_cell.geometry = -outer_bot | +outer_top
problem.cells.append(vacuum_cell)
print(vacuum_cell.mcnp_str())

# Step 3.2 Define cell importances

* Use [`cell.set_equal_importance`](https://www.montepy.org/en/stable/api/montepy.cells.html#montepy.cells.Cells.set_equal_importance) to do this easily
* Verify that `vacuum_cell` has an importance of 0

# Task: Set importances

In [ ]:
# Task: Set the importance of every cell except `vacuum_cell` to 1.0
problem.cells

for cell in problem.cells:
    print(cell.number, cell.material, cell.importance.neutron)
assert not vacuum_cell.importance.neutron, "The 'vacuum' cell has nonzero importance!"

# Step 4 Discretize Fuel Axially

## Steps
1. Shrink Lattice unit cell to be less than full length
2. Copy fill matrix multiple times to axially tile

# Step 4.1 Make new PZ surfaces
* Set unit cell height so multiple axial lattices can be inserted
  * Presenters will use 10 pellets. You may use any number you like

## Steps 
1. Clone a `pz` surface
2. Update top location

In [ ]:
# Task: Reduce the height of the unit cell.
NUM_AXIAL = None
internal_top = top.clone()
internal_top.location = None
internal_bot = bot.clone()
print(internal_top.mcnp_str())

## Step 4.2 find Lattice Cell
* Find cell with `lattice_type is not None`

In [ ]:
for cell in problem.cells:
    if cell.lattice_type is not None:
        lattice_cell = cell
        break
print(lattice_cell)

# Step 4.3 Update Geometry
* Need to have only 6 surfaces in a Hexahedra lattice
* Hard to remove surfaces from geometry definitions
* Recreate from scratch by iterating over the surfaces
    * If the `PX/PY` plane locations are negative we want the `+` side, and vise versa.
* Remember: need to list the top surface first in Z!
* [`Cell.surfaces`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.surfaces) is ordered by the original surface order

## Steps
1. Set aside a new geometry definition
2. Iterate over `Cell.surfaces`
    1. Exclude surfaces we don't want
    3. Make a half-space from surface based on sign
    4. Either set geometry as half-space or update it (with `&=`)
5. Add new `pz` surfaces to geometry definition
6. Update `lattice_cell.geometry`

In [ ]:
new_geom = None

for surf in lattice_cell.surfaces:
    if surf.surface_type not in [montepy.SurfaceType.PX, montepy.SurfaceType.PY]:
        continue
    if surf.location < 0:
        side = +surf
    else:
        side = -surf
    if new_geom is None:
        new_geom = side
    else:
        new_geom &= side
new_geom &= -internal_top & +internal_bot
lattice_cell.geometry = new_geom
print(lattice_cell.mcnp_str())

## Step 4.4: Expand fill matrix
* Use [numpy.repeat](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html) to accomplish this.
* Want to repeat in z, which equates to `axis=2`.

In [ ]:
# Extrude the lattice in the z-direction by using np.repeat()
fill_matrix = lattice_cell.fill.universes
fill_matrix = fill_matrix.repeat(NUM_AXIAL, axis=2)
lattice_cell.fill.universes = fill_matrix
print(lattice_cell.fill.universes.shape)

# Step 5: Updating `ksrc`

* Previously we have been using `0 0 0` as the default source position.
* Origin is now shifted.
* This site is on a material boundary and not possible.

## Note
--- 
* MontePy would be great for automating a reasonable `ksrc` definition
* Not the focus of this demonstration though

## Step 5.1: Find `ksrc`

* More difficult to find specific data inputs that are not numbered and fully supported
* Can use [`DataInput.prefix`](https://www.montepy.org/en/stable/api/montepy.data_inputs.data_input.html#montepy.data_inputs.data_input.DataInput.prefix)

### Steps 
* Iterate over `MCNP_Problem.data_inputs`
* Find the input with the appropriate `prefix`

In [ ]:
for data in problem.data_inputs:
    if data.prefix == "ksrc":
        ksrc = data
        break
ksrc

# Step 5.2: Update Values in `ksrc`
* MontePy doesn't support `ksrc` fully yet.
* Options:
    1. Delete this input, and create a new one
    2. Edit this in a slightly harder way using the syntax trees
* The "data", the list of values after the classifier, is stored in [`DataInput.data`](https://www.montepy.org/en/stable/api/montepy.data_inputs.data_input.html#montepy.data_inputs.data_input.DataInput.data)
* This is [`ListNode`](https://www.montepy.org/en/stable/api/montepy.input_parser.syntax_node.html#montepy.input_parser.syntax_node.ListNode) made up of [`ValueNode`](https://www.montepy.org/en/stable/api/montepy.input_parser.syntax_node.html#montepy.input_parser.syntax_node.ValueNode)s


# Steps
1. Find this `data` list node, and explore it
2. Grab the last node (the z coordinate) in the list
3. Update value to something more appropriate

In [ ]:
data = ksrc.data
print(data)
z_val = data[-1]
print(z_val)
z_val.value = 10.0
z_val

# Conclusion

* Write to file `models/w17_17_full_height.imcnp`

In [ ]:
problem.write_problem("models/w17_17_full_height.imcnp")

# Result plots

![xy slice of westinghouse 17x17](figs/w_17_17_axial_xy.png)
 * I swear this is a new plot, and not a copy from the last model

# Axial slice

## Questions?

![Axial slice of westinghouse 17x17](figs/w17x17_axial.png)

* The extent is not equal in `x` and `z`